**A Simple Dense Class**

In [2]:
import tensorflow as tf

class NaiveDense:# output = activation(dot(W, input) + b)
    def __init__(self, input_size, output_size, activation):
        self.activation = activation
        
        w_shape = (input_size, output_size)
        w_initial_value = tf.random.uniform(w_shape, minval=0, maxval=1e-1)
        self.W = tf.Variable(w_initial_value)
        
        b_shape = (output_size,)
        b_initial_value = tf.zeros(b_shape)
        self.b = tf.Variable(b_initial_value)
        
    def __call__(self, inputs):
        return self.activation(tf.matmul(inputs, self.W) + self.b)
    
    @property
    def weights(self):
        return [self.W, self.b]
    

**A Simple Sequential Class**

layer와 layer를 연결할 때 사용한다.

In [3]:
class NaiveSequential:
    def __init__(self, layers):
        self.layers = layers
        
    def __call__(self, inputs):
        x = inputs
        for layer in self.layers:
            x = layer(x)
        return x
    
    @property
    def weights(self):
        weights = []
        for layer in self.layers:
            weights += layer.weights
        return weights

In [4]:
model = NaiveSequential([
    NaiveDense(input_size=28*28, output_size=512, activation=tf.nn.relu),
    NaiveDense(input_size=512, output_size=10, activation=tf.nn.softmax)
])
assert len(model.weights) == 4

2022-01-09 22:06:56.862148: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-09 22:06:56.862812: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-09 22:06:56.868300: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-09 22:06:56.868985: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-09 22:06:56.869597: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from S

**A Batch Generator**

반복해서 mini-batch를 가져올 때 사용한다.

In [5]:
import math

class BatchGenerator:
    def __init__(self, images, labels, batch_size=128):
        assert len(images) == len(labels)
        self.index = 0
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.num_batches = math.ceil(len(images) / batch_size)
        
    def next(self):
        images = self.images[self.index : self.index+self.batch_size]
        labels = self.labels[self.index : self.index+self.batch_size]
        self.index += self.batch_size
        return images, labels

**One Training Step**

In [6]:
learning_rate = 1e-3
def update_weights(gradients, weights):
    for g, w in zip(gradients, weights):
        w.assign_sub(g * learning_rate) # w -= g * learing_rate

In [7]:
def one_training_step(model, images_batch, labels_batch):
    # Run the "forward pass"
    with tf.GradientTape() as tape:
        predictions = model(images_batch)
        # batch 마다 loss 계산
        per_sample_losses = tf.keras.losses.sparse_categorical_crossentropy(
            labels_batch, predictions)
        average_loss = tf.reduce_mean(per_sample_losses)
    gradients = tape.gradient(average_loss, model.weights)
    update_weights(gradients, model.weights)
    return average_loss

tensorflow를 사용할 때 실제로는 직접 모든 weight를 update 하지 않고 아래 코드 처럼 optimizers를 사용해서 weights를 update함

In [8]:
from tensorflow.keras import optimizers

optimizers = optimizers.SGD(learning_rate=1e-3)

def update_weights(gradients, weights):
    optimizers.apply_gradients(zip(gradients, weights))

## 2.5.3 The Full Training Loop

지금까지 구현했던 코드를 사용해서 전체 훈련 과정을 구현한다.

In [9]:
def fit(model, images, labels, epochs, batch_size=128):
    for epoch_counter in range(epochs):
        print(f"Epoch {epoch_counter}")
        batch_generator = BatchGenerator(images, labels, batch_size)
        for batch_counter in range(batch_generator.num_batches):
            images_batch, labels_batch = batch_generator.next()
            loss = one_training_step(model, images_batch, labels_batch)
            if batch_counter % 100 == 0:
                print(f"loss at batch {batch_counter}: {loss:.2f}")

In [10]:
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255

fit(model, train_images, train_labels, epochs=10, batch_size=128)

Epoch 0
loss at batch 0: 4.81
loss at batch 100: 2.24
loss at batch 200: 2.22
loss at batch 300: 2.12
loss at batch 400: 2.28
Epoch 1
loss at batch 0: 1.91
loss at batch 100: 1.89
loss at batch 200: 1.84
loss at batch 300: 1.74
loss at batch 400: 1.88
Epoch 2
loss at batch 0: 1.58
loss at batch 100: 1.59
loss at batch 200: 1.51
loss at batch 300: 1.44
loss at batch 400: 1.55
Epoch 3
loss at batch 0: 1.32
loss at batch 100: 1.35
loss at batch 200: 1.25
loss at batch 300: 1.22
loss at batch 400: 1.31
Epoch 4
loss at batch 0: 1.12
loss at batch 100: 1.17
loss at batch 200: 1.05
loss at batch 300: 1.06
loss at batch 400: 1.13
Epoch 5
loss at batch 0: 0.97
loss at batch 100: 1.03
loss at batch 200: 0.91
loss at batch 300: 0.94
loss at batch 400: 1.01
Epoch 6
loss at batch 0: 0.86
loss at batch 100: 0.92
loss at batch 200: 0.81
loss at batch 300: 0.84
loss at batch 400: 0.91
Epoch 7
loss at batch 0: 0.78
loss at batch 100: 0.84
loss at batch 200: 0.73
loss at batch 300: 0.77
loss at batch 40

In [11]:
import numpy as np

predictions = model(test_images)
predictions = predictions.numpy()
predicted_labels = np.argmax(predictions, axis=1)
matches = predicted_labels == test_labels